In [3]:
import os
import sys
sys.path.append('/Users/wangqiqian/Desktop/TrafficTDApython/')

current_dir_path = os.getcwd()
current_file_path = os.path.abspath(current_dir_path)
current_dir_path = os.path.dirname(current_file_path)
parent_dir_path = os.path.dirname(current_dir_path)

# os.chdir(current_dir_path + '\\ultils')
os.chdir(current_dir_path + '/ultils')
print(current_dir_path)

import time
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.cluster import AgglomerativeClustering

from functions import *
from chi import *
from regressionP import *
from models import *
from utils_v3 import *

os.chdir('/Users/wangqiqian/Desktop/TrafficTDApython/')
# os.chdir('C:/Users/USER/Desktop/TrafficTDApython/')

/Users/wangqiqian/Desktop/TrafficTDApython


In [4]:
data1 = pd.read_csv("Data/NPA_TMA2_1.csv", low_memory=False)[:-2]
data2 = pd.read_csv("Data/NPA_TMA2_2.csv", low_memory=False)[:-2]
data3 = pd.read_csv("Data/NPA_TMA2_3.csv", low_memory=False)[:-2]
data4 = pd.read_csv("Data/NPA_TMA2_4_new.csv", low_memory=False)[:-2]
data5 = pd.read_csv("Data/NPA_TMA2_5.csv", low_memory=False)[:-2]
data6 = pd.read_csv("Data/NPA_TMA2_6_new.csv", low_memory=False)[:-2]
data7 = pd.read_csv("Data/NPA_TMA2_7.csv", low_memory=False)[:-2]
data8 = pd.read_csv("Data/NPA_TMA2_8.csv", low_memory=False)[:-2]
data9 = pd.read_csv("Data/NPA_TMA2_9.csv", low_memory=False)[:-2]
data10 = pd.read_csv("Data/NPA_TMA2_10.csv", low_memory=False)[:-2]

dataA2 = pd.concat([data1, data2, data3, data4, data5, data6, data7, data8, data9, data10], ignore_index=True)
# dataA2 = pd.concat([data1, data2, data3, data4, data5], ignore_index=True)

dataA1 = pd.read_csv("Data/NPA_TMA1_V3.csv")[:-2]

In [5]:
select_lst = [
    '天候名稱', '光線名稱',
    '道路類別-第1當事者-名稱', '速限-第1當事者', '道路型態大類別名稱', '道路型態子類別名稱',
    '事故位置大類別名稱', '事故位置子類別名稱', '路面狀況-路面鋪裝名稱', '路面狀況-路面狀態名稱', '路面狀況-路面缺陷名稱',
    '道路障礙-障礙物名稱', '道路障礙-視距品質名稱', '道路障礙-視距名稱', '號誌-號誌種類名稱', '號誌-號誌動作名稱',
    '車道劃分設施-分向設施大類別名稱', '車道劃分設施-分向設施子類別名稱', '車道劃分設施-分道設施-快車道或一般車道間名稱',
    '車道劃分設施-分道設施-快慢車道間名稱', '車道劃分設施-分道設施-路面邊線名稱', '事故類型及型態大類別名稱',
    '事故類型及型態子類別名稱', '死亡受傷人數',
    '肇因研判子類別名稱-個別'
    # '經度', '緯度'
]

def preprocess(input_data, select_lst):
    
    sample_data = input_data[input_data['當事者順位'] == 1].reset_index(drop=True, inplace=False)
    sample_data = sample_data[sample_data['發生月份'] < 11]
    dataA = sample_data[select_lst]
    
    # 將資料分出死亡和受傷，合併到原本的資料後去除多餘的死亡受傷人數
    death_injury_data = split_death_injury(dataA['死亡受傷人數'])
    dist_df = pd.concat([dataA, death_injury_data], axis=1)
    dist_df.drop(columns=['死亡受傷人數'], inplace=True)
    
    return dist_df

dist_dfA1 = preprocess(dataA1, select_lst)
dist_dfA2 = preprocess(dataA2, select_lst)
# Concat
rbind_data = pd.concat([dist_dfA1, dist_dfA2], axis=0, ignore_index=True)

# Sample
# rbind_data = rbind_data.sample(frac=0.3, random_state=43)
# sample_data_to_map.reset_index(drop=True, inplace=True)

# assert rbind_data.shape[0] == dist_dfA1.shape[0] + dist_dfA2.shape[0]
# Dummy
rbind_data["速限-第1當事者"] = rbind_data["速限-第1當事者"].astype(str)
dummy_data = pd.get_dummies(rbind_data)

print('dummy_data:', dummy_data.shape)

data_to_map = dummy_data.drop(['死亡', '受傷'], axis=1)
mapper_numpy = data_to_map.to_numpy()

dummy_data: (312760, 332)


### Multiple correspondence analysis (MCA)
多重對應分析是專門針對dummy來進行降維的，所以使用它作為主要方法沒有問題

In [9]:
# import prince

# mca = prince.MCA(
#     one_hot=False,
#     n_components=8,
#     n_iter=30,
#     copy=True,
#     check_input=True,
#     random_state=42
# )

# mca.fit(data_to_map)
# lens = mca.transform(data_to_map)

# print(mca.eigenvalues_summary)

          eigenvalue % of variance % of variance (cumulative)
component                                                    
0              0.238         1.87%                      1.87%
1              0.167         1.31%                      3.17%
2              0.152         1.19%                      4.37%
3              0.135         1.06%                      5.43%
4              0.119         0.93%                      6.36%
5              0.109         0.86%                      7.22%
6              0.102         0.80%                      8.01%
7              0.092         0.72%                      8.74%


1. lens：以component為8對所有資料做MCA

In [13]:
# lens.to_csv('CalculatedData/lens.csv', index=False)

lens = pd.read_csv("CalculatedData/lens.csv")
lens

,0,1,2,3,4,5,6,7
0,0.882732,0.269135,-0.369187,-1.251236,0.596204,-0.723201,0.761794,0.185655
1,-0.356824,-0.029570,0.221838,0.045091,0.160383,-0.046000,-0.061379,0.263801
2,0.855796,0.008016,-0.108392,0.103317,-0.148224,-0.148898,0.804930,-0.291473
3,-0.838256,0.034320,-0.281365,0.211452,0.246303,0.132811,0.039984,-0.006935
4,0.815752,0.044299,-0.419932,0.268203,0.001200,-0.071992,0.962080,-0.307298
...,...,...,...,...,...,...,...,...
312755,0.697598,-0.056682,0.483826,-0.197071,0.136961,-0.053113,0.195439,0.582318
312756,0.917793,-0.038410,0.407264,-0.199388,0.941853,0.464400,0.710561,0.128872
312757,-0.342071,-0.037612,0.283167,-0.090342,-0.236115,0.023882,0.188919,-0.131607
312758,-0.095553,-0.046641,0.353162,-0.203712,-0.181460,0.020557,0.630802,-0.140866


# Find the best mapper

In [6]:
from Version3.tdamapper.core import MapperAlgorithm
from Version3.tdamapper.cover import CubicalCover
from Version3.tdamapper.clustering import FailSafeClustering
from Version3.tdamapper.plot import MapperLayoutInteractive

New version
New version


In [ ]:
mapper_algo = MapperAlgorithm(
    cover=CubicalCover(
        n_intervals=interval,
        overlap_frac=overlap / 10
    ),
    clustering=FailSafeClustering(
        AgglomerativeClustering(
        n_clusters=2,
        linkage='single'
        )
    )
)
mapper_info = mapper_algo.fit_transform(mapper_numpy, lens)

In [7]:
detailed_results = []

for overlap in range(3, 6):
# for overlap in range(5, 6):
    silhouette_for_intervals = []

    # for interval in range(5, 11):
    for interval in range(10, 11):
        print(f'overlap: {overlap}, interval: {interval}')
        mapper_algo = MapperAlgorithm(
            cover=CubicalCover(
                n_intervals=interval,
                overlap_frac=overlap / 10
            ),
            clustering=FailSafeClustering(
                AgglomerativeClustering(
                n_clusters=2,
                linkage='single'
                )
            )
        )

        mapper_info = mapper_algo.fit_transform(mapper_numpy, lens)
        silhouette_for_intervals.append(mapper_info[1])

        detailed_results.append({
            "overlap": overlap,
            "interval": interval,
            "silhouette": mapper_info[1],
            "mapper_info": mapper_info
        })

overlap: 3, interval: 10


NameError: name 'lens' is not defined

1. full_grid：使用Interval優化的前兩個月grid search
2. full_grid_sample：抽樣30%資料且沒有使用Interval優化
3. full_grid_sample_r2：和第二點一樣但是第二回合
4. full_grid_fulldata_o5i6：整筆資料的拓樸使用覆蓋為5區間數為6
5. full_grid_fulldata_o345i10：區間數固定為10的情況下覆蓋為345的資料

In [ ]:
# detailed_results_df = pd.DataFrame(detailed_results)
# detailed_results_df.to_csv("/Users/wangqiqian/Desktop/TrafficTDApython/Version3/GridSearch/full_grid_fulldata_o345i10.csv", index=False)

# output_file = '/Users/wangqiqian/Desktop/TrafficTDApython/Version3/GridSearch/full_grid_fulldata_o345i10.pkl'
# with open(output_file, 'wb') as f:
#     pickle.dump(detailed_results_df, f)

# print(f"Results saved to {output_file}")